In [ ]:
from __future__ import print_function
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pyemma
import mdshare

# import some functions which should not clutter the notebook
import shortcuts_thermo as shortcuts

# figure size parameters
pw = 6
ph = 0.75 * pw

# Example - one-dimensional asymmetric double well potential

We start by looking at the stationary distribution and free energy profile which are available analytically.

In [ ]:
adw_x, adw_f, adw_pi = shortcuts.adw_reference(-1, 5, 100)

fig, ax = plt.subplots(1, 2, figsize=(2 * pw, ph))
ax[0].plot(adw_x, adw_pi, linewidth=3, color='black')
ax[0].set_ylabel(r"$\pi(x)$", fontsize=20)
ax[0].semilogy()
ax[1].plot(adw_x, adw_f, linewidth=3, color='black')
ax[1].set_ylabel(r"$f(x)$ / kT", fontsize=20)
for _ax in ax:
    _ax.set_xlabel(r"$x$ / a.u.", fontsize=20)
    _ax.tick_params(labelsize=15)
fig.tight_layout()

## Multi-temperature (simulated tempering data)


$$b^{(i)}(\mathbf{x}) = \left( \frac{1}{\text{k}_\text{B} T^{(i)}} - \frac{1}{\text{k}_\text{B} T^{\circ}} \right) U(\mathbf{x}) = \left( \frac{1}{\text{k}_\text{B} T^{(i)}} - \frac{1}{\text{k}_\text{B} T^{\circ}} \right) \text{k}_\text{B} T^{(j)} u^{(j)}(\mathbf{x})$$

Let's plot the stationary distributions and free energy profiles for four different kT values:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * pw, ph))
# plot the thermodynamic ground/unbiased state (kT=1.0)
ax[0].plot(adw_x, adw_pi, linewidth=3, color='black', label='kT=1.0')
ax[1].plot(adw_x, adw_f, linewidth=3, color='black', label='kT=1.0')
# plot the kT=2.0 case
_, adw_f2, adw_pi2 = shortcuts.adw_reference(adw_x[0], adw_x[-1], adw_x.shape[0], kT=2.0)
ax[0].plot(adw_x, adw_pi2, linewidth=3, color='blue', label='kT=2.0')
ax[1].plot(adw_x, adw_f2, linewidth=3, color='blue', label='kT=2.0')
# plot the kT=3.5 case
_, adw_f2, adw_pi2 = shortcuts.adw_reference(adw_x[0], adw_x[-1], adw_x.shape[0], kT=3.5)
ax[0].plot(adw_x, adw_pi2, linewidth=3, color='green', label='kT=3.5')
ax[1].plot(adw_x, adw_f2, linewidth=3, color='green', label='kT=3.5')
# plot the kT=7.5 case
_, adw_f2, adw_pi2 = shortcuts.adw_reference(adw_x[0], adw_x[-1], adw_x.shape[0], kT=7.5)
ax[0].plot(adw_x, adw_pi2, linewidth=3, color='red', label='kT=7.5')
ax[1].plot(adw_x, adw_f2, linewidth=3, color='red', label='kT=7.5')
# finish the figure
ax[0].set_ylabel(r"$\pi^{(j)}(x)$", fontsize=20)
ax[0].semilogy()
ax[0].legend(loc=8, fontsize=12, fancybox=True, framealpha=0.5)
ax[1].set_ylabel(r"$f^{(j)}(x) - f^{(j)}$ / kT", fontsize=20)
ax[1].legend(loc=9, fontsize=12, fancybox=True, framealpha=0.5)
for _ax in ax:
    _ax.set_xlabel(r"$x$ / a.u.", fontsize=20)
    _ax.tick_params(labelsize=15)
fig.tight_layout()

``pyemma.thermo`` provides an API function for this simulation type:

```python
def estimate_multi_temperature(
    energy_trajs, temp_trajs, dtrajs,
    energy_unit='kcal/mol', temp_unit='K', reference_temperature=None,
    maxiter=10000, maxerr=1.0E-15, save_convergence_info=0,
    estimator='wham', lag=1, dt_traj='1 step', init=None):
    ...
```

First step: import the data from 20 precomputed trajectories...

In [ ]:
with np.load(mdshare.load('pyemma-tutorial-mt-data.npz', working_directory='data')) as fh:
    adw_st_conf_trajs = [fh['conf_traj_%03d.npy' % i] for i in range(20)]
    adw_st_temp_trajs = [fh['temp_traj_%03d.npy' % i] for i in range(20)]
    adw_st_energy_trajs = [fh['energy_traj_%03d.npy' % i] for i in range(20)]

For one trajectory, plot all three types of data.

In [ ]:
adw_st_index = 4
fig, ax = plt.subplots(3, 1, figsize=(2 * pw, 3 * ph), sharex=True)
ax[0].plot(adw_st_conf_trajs[adw_st_index][:, 0], '-x')
ax[1].plot(adw_st_temp_trajs[adw_st_index], '-x')
ax[2].plot(adw_st_energy_trajs[adw_st_index], '-x')
ax[2].set_xlabel(r"time / steps", fontsize=20)
ax[0].set_ylabel(r"configuration state / a.u.", fontsize=20)
ax[1].set_ylabel(r"heat bath / kT", fontsize=20)
ax[2].set_ylabel(r"energy / kT", fontsize=20)
for _ax in ax:
    _ax.tick_params(labelsize=15)
fig.tight_layout()

Second step: run a clustering algorithm on the configuration trajectories...

In [ ]:
adw_st_cluster = pyemma.coordinates.cluster_regspace(adw_st_conf_trajs, max_centers=500, dmin=0.2)

Third step: run ``WHAM`` and ``dTRAM`` estimations using the ``estimate_multi_temperature`` API function...

In [ ]:
adw_st_estimator = pyemma.thermo.estimate_multi_temperature(
    adw_st_energy_trajs, adw_st_temp_trajs, adw_st_cluster.dtrajs,
    energy_unit='kT', temp_unit='kT',
    maxiter=10000, maxerr=1.0E-14, save_convergence_info=1, estimator='dtram')

In [ ]:
pyemma.plots.plot_convergence_info(adw_st_estimator)

Fourth step: plot the free energies ``f`` and ``f_therm``...

In [ ]:
adw_st_x, adw_st_f = shortcuts.adw_match_reference_to_binning(adw_st_conf_trajs, adw_st_cluster.clustercenters)

fig, ax = plt.subplots(1, 2, figsize=(2 * pw, ph))
ax[0].plot(
    adw_st_cluster.clustercenters[adw_st_estimator.active_set, 0], adw_st_estimator.f, 's', markersize=10, label=adw_st_estimator.name)
ax[0].plot(adw_st_x, adw_st_f, '-*', linewidth=2, markersize=9, color='black', label='Reference')
ax[0].set_xlabel(r"configuration state", fontsize=20)
ax[0].set_ylabel(r"f / kT", fontsize=20)
ax[0].legend(loc=0, fontsize=10, fancybox=True, framealpha=0.5)
ax[1].plot(adw_st_estimator.temperatures, adw_st_estimator.f_therm, 's', markersize=10, label=adw_st_estimator.name)
ax[1].set_ylabel(r"f_therm / kT", fontsize=20)
ax[1].legend(loc=4, fontsize=10, fancybox=True, framealpha=0.5)
ax[1].set_xlabel(r"kT", fontsize=20)
for _ax in ax:
    _ax.tick_params(labelsize=15)
fig.tight_layout()

Plot the stationay distribution of the thermodynamic ground state.

In [ ]:
plt.figure(figsize=(pw, ph))
plt.plot(
    adw_st_cluster.clustercenters[adw_st_estimator.active_set, 0], adw_st_estimator.pi, 's', markersize=10)
plt.xlabel(r"configuration state", fontsize=20)
plt.ylabel(r"pi", fontsize=20)
plt.ylim([1.0E-11, 1.0])
plt.yscale('log')

Last step: obtain some kinetic information.
In this case, we run ``PCCA`` on the ``MSM`` of the thermodynamic ground state and compute mean first passage times between the two metastable sets...

In [ ]:
# The MSM of the unbiased ensemble can be accessed via dtram_estiamtor.msm
unbiased_msm = adw_st_estimator.msm

# We can do all the usual MSM analyses now, e. g. coarse-graining with PCCA and computing MFPTs.
pcca = unbiased_msm.pcca(2)

print("MFPT[blue->green] = %7.1f steps" %  unbiased_msm.mfpt(pcca.metastable_sets[0], pcca.metastable_sets[1]))
print("MFPT[green->blue] = %7.1f steps" %  unbiased_msm.mfpt(pcca.metastable_sets[1], pcca.metastable_sets[0]))

plt.plot(adw_st_x, adw_st_f, '-*', linewidth=2, markersize=9, color='black')
plt.scatter(
    adw_st_cluster.clustercenters[unbiased_msm.active_set, 0],
    -np.log(unbiased_msm.stationary_distribution[unbiased_msm.active_set]),
    s=120, c=pcca.metastable_assignment, cmap=mpl.cm.brg)

plt.xlabel(r"configuration state", fontsize=20)
plt.ylabel(r"f / kT", fontsize=20)
plt.tick_params(labelsize=15)
plt.xlim([-1, 5])
plt.ylim([0, 12])